In [1]:
#@title Installs

!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 1.3 MB/s eta 0:00:00


In [2]:
#@title Imports

#for Layer
from abc import ABC, abstractmethod
import math
import torch
import torch.autograd
import torch.nn.init
import torch.nn.functional as F
from torch.nn.parameter import Parameter
#from torch._jit_internal import weak_module, weak_script_method

#for util.operation
from torch.utils.data import Dataset, Subset

#for VCL model
import torch.nn as nn
import math

#for Coreset
import torch.utils.data as data
from copy import deepcopy
import numpy as np
import torch.optim as optim
from tqdm import tqdm
from random import shuffle

#for run_task
from torch.utils.data import DataLoader
#NEGLECTED: from util.plot_autograd import make_dot

#for Permuted MNIST experiments
from tensorboardX import SummaryWriter
from torchvision.datasets import MNIST
from torchvision.transforms import Compose
from torch.utils.data import ConcatDataset
#from datetime import datetime

#for replay
from torch.autograd import grad
from torch.nn.utils import parameters_to_vector

#misc
import random
import pickle

#for logging results
import json
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Running on device", DEVICE)

Running on device cuda


In [6]:
#@title util.operation

#taken partially from https://github.com/NixGD/variational-continual-learning public repo (w minor edits)

def class_accuracy(pred: torch.Tensor, true: torch.Tensor) -> float:
    """
    Computes the percentage class accuracy of the predictions, given the correct
    class labels.

    Args:
        pred: the class predictions made by a model
        true: the ground truth classes of the sample
    Returns:
        Classification accuracy of the predictions w.r.t. the ground truth labels
    """
    #EDITED to put pred and true on GPU
    return 100 * (pred.int().to(device=DEVICE) == true.int().to(device=DEVICE)).sum().item() / len(true)


def kl_divergence(z_posterior_means, z_posterior_log_std, z_prior_mean=0.0, z_prior_log_std=0.0):
    """ Computes KL(z_posterior, z_prior) """
    # code adapted from author implementation at
    # https://github.com/nvcuong/variational-continual-learning/blob/master/dgm/alg/helper_functions.py
    z_prior_means = torch.full_like(z_posterior_means, z_prior_mean)
    z_prior_log_stds = torch.full_like(z_posterior_log_std, z_prior_log_std)

    prior_precision = torch.exp(torch.mul(z_prior_log_stds, -2))
    kl = 0.5 * ((z_posterior_means - z_prior_means) ** 2) * prior_precision - 0.5
    kl += z_prior_log_stds - z_posterior_log_std
    kl += 0.5 * torch.exp(2 * z_posterior_log_std - 2 * z_prior_log_stds)
    return torch.sum(kl, dim=(1,))


def bernoulli_log_likelihood(x_observed, x_reconstructed, epsilon=1e-8) -> torch.Tensor:
    """
    For observed batch of data x, and reconstructed data p (we view p as a
    probability of a pixel being on), computes a tensor of dimensions
    [batch_size] representing the log likelihood of each data point in the batch.
    """
    # broken into steps because some log probabilities are extremely low and cause NaNs to appear
    # as a hacky solution, we replace NaNs with a log probability of 10**-8 as an intermediate step
    prob = torch.mul(torch.log(x_reconstructed + epsilon), x_observed)
    inv_prob = torch.mul(torch.log(1 - x_reconstructed + epsilon), 1 - x_observed)
    inv_prob[inv_prob != inv_prob] = epsilon

    return torch.sum(torch.add(prob, inv_prob), dim=(1,))


def normal_with_reparameterization(means: torch.Tensor, log_stds: torch.Tensor, device='cpu') -> torch.Tensor:
    return torch.add(means, torch.mul(torch.exp(log_stds), torch.randn_like(means)))


def concatenate_flattened(tensor_list) -> torch.Tensor:
    """
    Given list of tensors, flattens each and concatenates their values.
    """
    return torch.cat([torch.reshape(t, (-1,)) for t in tensor_list])


def task_subset(data: Dataset, task_ids: torch.Tensor, task: int,) -> torch.Tensor:
    idx_list = torch.arange(0, len(task_ids))[task_ids == task]
    return Subset(data, idx_list)


In [7]:
#@title util.transforms

#COPIED FROM https://github.com/NixGD/variational-continual-learning repo

class Flatten(object):
    """ Transforms a PIL image to a flat numpy array. """
    def __init__(self):
        pass

    def __call__(self, sample):
        return np.array(sample, dtype=np.float32).flatten()


class Scale(object):
    """Scale images down to have [0,1] float pixel values"""
    def __init__(self, max_value=255):
        self.max_value = max_value

    def __call__(self, sample):
        return sample / self.max_value


class Permute(object):
    """ Apply a fixed permutation to the pixels in the image. """
    def __init__(self, permutation):
        self.permutation = permutation

    def __call__(self, sample):
        return sample[self.permutation]

In [8]:
#@title VCL model

#taken from https://github.com/NixGD/variational-continual-learning with edits to accomodate replay buffer

TRAIN_NUM_SAMPLES = 10
TEST_NUM_SAMPLES = 50
EPSILON = 1e-8  # Small value to avoid divide-by-zero and log(zero) problems


class DiscriminativeVCL(nn.Module):
    """
    A Bayesian multi-head neural network which updates its parameters using
    variational inference.
    """

    def __init__(self, in_size: int, out_size: int, layer_width: int,
                 n_hidden_layers: int, n_heads: int, initial_posterior_var: int):
        super().__init__()
        self.input_size = in_size
        self.out_size = out_size
        self.n_hidden_layers = n_hidden_layers
        self.layer_width = layer_width
        self.n_heads = n_heads

        print("Number of heads:", n_heads)

        self.prior, self.posterior = None, None
        self.head_prior, self.head_posterior = None, None

        self._init_variables(initial_posterior_var)

    def to(self, *args, **kwargs):
        """
        Our prior tensors are registered as buffers but the way we access them
        indirectly (through tuple attributes on the model) is causing problems
        because when we use `.to()` to move the model to a new device, the prior
        tensors get moved (because they're registered as buffers) but the
        references in the tuples don't get updated to point to the new moved
        tensors. This has no effect when running just on a cpu but breaks the
        model when trying to run on a gpu. There are a million nicer ways of
        working around this problem, but for now the easiest thing is to do
        this: override the `.to()` method and manually update our references to
        prior tensors.
        """
        self = super().to(*args, **kwargs)
        (prior_w_means, prior_w_log_vars), (prior_b_means, prior_b_log_vars) = self.prior
        prior_w_means = [t.to(*args, **kwargs) for t in prior_w_means]
        prior_w_log_vars = [t.to(*args, **kwargs) for t in prior_w_log_vars]
        prior_b_means = [t.to(*args, **kwargs) for t in prior_b_means]
        prior_b_log_vars = [t.to(*args, **kwargs) for t in prior_b_log_vars]
        self.prior = (prior_w_means, prior_w_log_vars), (prior_b_means, prior_b_log_vars)
        (head_prior_w_means, head_prior_w_log_vars), (head_prior_b_means, head_prior_b_log_vars) = self.head_prior
        head_prior_w_means = [t.to(*args, **kwargs) for t in head_prior_w_means]
        head_prior_w_log_vars = [t.to(*args, **kwargs) for t in head_prior_w_log_vars]
        head_prior_b_means = [t.to(*args, **kwargs) for t in head_prior_b_means]
        head_prior_b_log_vars = [t.to(*args, **kwargs) for t in head_prior_b_log_vars]
        self.head_prior = (head_prior_w_means, head_prior_w_log_vars), (head_prior_b_means, head_prior_b_log_vars)
        return self

    def forward(self, x, head):
        """ Forward pass of the model on an input. """
        # sample layer parameters from posterior distribution
        (w_means, w_log_vars), (b_means, b_log_vars) = self.posterior
        (head_w_means, head_w_log_vars), (head_b_means, head_b_log_vars) = self.head_posterior
        sampled_layers = self._sample_parameters(w_means, b_means, w_log_vars, b_log_vars)
        sampled_head_layers = self._sample_parameters(head_w_means, head_b_means, head_w_log_vars, head_b_log_vars)

        # Apply each layer with its sampled weights and biases
        for weight, bias in sampled_layers:
            #print(f'applying W with shape : {(weight.size(0),weight.size(1))}')#DEBUG
            x = F.relu(x @ weight + bias)

        head_weight, head_bias = list(sampled_head_layers)[head]
        #print(f'applying W with shape : {(head_weight.size(0),head_weight.size(1))}')#DEBUG
        x = x @ head_weight + head_bias

        return x

    def vcl_loss(self, x, y, head, task_size, num_samples=TRAIN_NUM_SAMPLES) -> torch.Tensor:
        """
        Returns the loss of the model, as described in equation 4 of the Variational
        Continual Learning paper (https://arxiv.org/abs/1710.10628).
        """
        return self._calculate_kl_term(head).cpu() / task_size - self._log_prob(x, y, head, num_samples)

    def point_estimate_loss(self, x, y, head=0):
        """
        Returns a loss defined in terms of a simplified forward pass that
        doesn't use sampling, and so uses the posterior means but not the
        variances. Used as part of model initialisation to optimise the
        posterior means to point-estimates for the first head.
        """
        (w_means, _), (b_means, _) = self.posterior
        (head_w_means, _), (head_b_means, _) = self.head_posterior

        for weight, bias in zip(w_means, b_means):
            x = F.relu(x @ weight + bias)

        x = x @ head_w_means[head] + head_b_means[head]

        return nn.CrossEntropyLoss()(x, y)

    def prediction(self, x, head, num_samples=TEST_NUM_SAMPLES):
        """Returns an integer between 0 and self.out_size"""
        outputs = torch.empty(num_samples, len(x), self.out_size)
        for i in range(num_samples):
            outputs[i] = nn.Softmax(dim=1)(self.forward(x, head))

        predictions = outputs.mean(dim=0)

        return torch.argmax(predictions, dim=1)

    def reset_for_new_task(self, head):
        """
        Called after completion of a task, to reset state for the next task
        """
        # Set the value of the prior to be the current value of the posterior
        (prior_w_means, prior_w_log_vars), (prior_b_means, prior_b_log_vars) = self.prior
        (post_w_means, post_w_log_vars), (post_b_means, post_b_log_vars) = self.posterior
        for i in range(self.n_hidden_layers):
            prior_w_means[i].data.copy_(post_w_means[i].data)
            prior_w_log_vars[i].data.copy_(post_w_log_vars[i].data)
            prior_b_means[i].data.copy_(post_b_means[i].data)
            prior_b_log_vars[i].data.copy_(post_b_log_vars[i].data)

        # set the value of the head prior to be the current value of the posterior
        (head_prior_w_means, head_prior_w_log_vars), (head_prior_b_means, head_prior_b_log_vars) = self.head_prior
        (head_posterior_w_means, head_posterior_w_log_vars), (head_posterior_b_means, head_posterior_b_log_vars) = self.head_posterior
        head_prior_w_means[head].data.copy_(head_posterior_w_means[head].data)
        head_prior_w_log_vars[head].data.copy_(head_posterior_w_log_vars[head].data)
        head_prior_b_means[head].data.copy_(head_posterior_b_means[head].data)
        head_prior_b_log_vars[head].data.copy_(head_posterior_b_log_vars[head].data)

    def _calculate_kl_term(self, head):
        """
        Calculates and returns the KL divergence of the new posterior and the previous
        iteration's posterior. See equation L3, slide 14.
        """
        # Prior
        ((prior_w_means, prior_w_log_vars), (prior_b_means, prior_b_log_vars)) = self.prior
        ((head_prior_w_means, head_prior_w_log_vars),
         (head_prior_b_means, head_prior_b_log_vars)) = self.head_prior

        prior_means = concatenate_flattened(
            prior_w_means + head_prior_w_means[head:head+1] +
            prior_b_means + head_prior_b_means[head:head+1])
        prior_log_vars = concatenate_flattened(
            prior_w_log_vars + head_prior_w_log_vars[head:head+1] +
            prior_b_log_vars + head_prior_b_log_vars[head:head+1])
        prior_vars = torch.exp(prior_log_vars)

        # Posterior
        ((post_w_means, post_w_log_vars), (post_b_means, post_b_log_vars)) = self.posterior
        ((head_post_w_means, head_post_w_log_vars),
         (head_post_b_means, head_post_b_log_vars)) = self.head_posterior

        post_means = concatenate_flattened(
            post_w_means + head_post_w_means[head:head+1] +
            post_b_means + head_post_b_means[head:head+1])
        post_log_vars = concatenate_flattened(
            post_w_log_vars + head_post_w_log_vars[head:head+1] +
            post_b_log_vars + head_post_b_log_vars[head:head+1])
        post_vars = torch.exp(post_log_vars)

        # Calculate KL for individual normal distributions over parameters
        kl_elementwise = \
            post_vars / (prior_vars + EPSILON) + \
            torch.pow(prior_means - post_means, 2) / (prior_vars + EPSILON) \
            - 1 + prior_log_vars - post_log_vars

        # Sum KL over all parameters
        return 0.5 * kl_elementwise.sum()

    def _log_prob(self, x, y, head, num_samples):

        outputs = []
        for i in range(num_samples):
            outputs.append(self.forward(x, head))
        return - nn.CrossEntropyLoss()(torch.cat(outputs).to(DEVICE), y.repeat(num_samples).view(-1).to(DEVICE)) #EDITED to add "to(DEVICE)"

    def _sample_parameters(self, w_means, b_means, w_log_vars, b_log_vars):
        # sample weights and biases from normal distributions
        sampled_weights, sampled_bias = [], []
        for layer_n in range(len(w_means)):
            w_epsilons = torch.randn_like(w_means[layer_n])
            b_epsilons = torch.randn_like(b_means[layer_n])
            sampled_weights.append(w_means[layer_n] + w_epsilons * torch.exp(0.5 * w_log_vars[layer_n]))
            sampled_bias.append(b_means[layer_n] + b_epsilons * torch.exp(0.5 * b_log_vars[layer_n]))
        return zip(sampled_weights, sampled_bias)

    def _init_variables(self, initial_posterior_var):
        """
        Initializes the model's prior and posterior weights / biases to their initial
        values. This method is called once on model creation. The model prior is registered
        as a persistent part of the model state which should not be modified, while the
        initial posterior is registered as a model parameter to be optimized.

        To avoid negative variances, we do not store parameter variances directly; instead
        we store the logarithm of each variance, and apply the exponential as needed in the
        forward pass.
        """
        # The initial prior over the parameters has zero mean, unit variance (i.e. log variance 0)
        prior_w_means = [torch.zeros(self.input_size, self.layer_width)] + \
                        [torch.zeros(self.layer_width, self.layer_width) for _ in range(self.n_hidden_layers - 1)]
        prior_w_log_vars = [torch.zeros_like(t) for t in prior_w_means]
        prior_b_means = [torch.zeros(self.layer_width) for _ in range(self.n_hidden_layers)]
        prior_b_log_vars = [torch.zeros_like(t) for t in prior_b_means]

        self.prior = ((prior_w_means, prior_w_log_vars), (prior_b_means, prior_b_log_vars))

        head_prior_w_means = [torch.zeros(self.layer_width, self.out_size) for t in range(self.n_heads)]
        head_prior_w_log_vars = [torch.zeros_like(t) for t in head_prior_w_means]
        head_prior_b_means = [torch.zeros(self.out_size) for t in range(self.n_heads)]
        head_prior_b_log_vars = [torch.zeros_like(t) for t in head_prior_b_means]

        self.head_prior = ((head_prior_w_means, head_prior_w_log_vars), (head_prior_b_means, head_prior_b_log_vars))

        empty_parameter_like = lambda t: nn.Parameter(torch.empty_like(t, requires_grad=True))

        posterior_w_means = [empty_parameter_like(t) for t in prior_w_means]
        posterior_w_log_vars = [empty_parameter_like(t) for t in prior_w_log_vars]
        posterior_b_means = [empty_parameter_like(t) for t in prior_b_means]
        posterior_b_log_vars = [empty_parameter_like(t) for t in prior_b_log_vars]

        self.posterior = ((posterior_w_means, posterior_w_log_vars), (posterior_b_means, posterior_b_log_vars))

        head_posterior_w_means = [empty_parameter_like(t) for t in head_prior_w_means]
        head_posterior_w_log_vars = [empty_parameter_like(t) for t in head_prior_w_log_vars]
        head_posterior_b_means = [empty_parameter_like(t) for t in head_prior_b_means]
        head_posterior_b_log_vars = [empty_parameter_like(t) for t in head_prior_b_log_vars]

        self.head_posterior = \
            ((head_posterior_w_means, head_posterior_w_log_vars),
             (head_posterior_b_means, head_posterior_b_log_vars))

        # Initialise the posterior means with a normal distribution. Note that
        # prior to training we will run a procedure to optimise these values to
        # point-estimates of the parameters for the first task.
        for t in posterior_w_means + posterior_b_means + head_posterior_w_means + head_posterior_b_means:
            torch.nn.init.normal_(t, mean=0, std=0.1)

        # Initialise the posterior variances with the given constant value.
        for t in posterior_w_log_vars + posterior_b_log_vars + head_posterior_w_log_vars + head_posterior_b_log_vars:
            torch.nn.init.constant_(t, math.log(initial_posterior_var))

        # Finally, we register the prior and the posterior with the nn.Module.
        # The prior values are registered as buffers, which indicates to PyTorch
        # that they represent persistent state which should not be updated by
        # the optimizer. The posteriors are registered as parameters, which on
        # the other hand are to be modified by the optimizer.
        for i in range(self.n_hidden_layers):
            self.register_buffer("prior_w_means_" + str(i), prior_w_means[i])
            self.register_buffer("prior_w_log_vars_" + str(i), prior_w_log_vars[i])
            self.register_buffer("prior_b_means_" + str(i), prior_b_means[i])
            self.register_buffer("prior_b_log_vars_" + str(i), prior_b_log_vars[i])

        for i in range(self.n_heads):
            self.register_buffer("head_prior_w_means_" + str(i), head_prior_w_means[i])
            self.register_buffer("head_prior_w_log_vars_" + str(i), head_prior_w_log_vars[i])
            self.register_buffer("head_prior_b_means_" + str(i), head_prior_b_means[i])
            self.register_buffer("head_prior_b_log_vars_" + str(i), head_prior_b_log_vars[i])

        for i in range(self.n_hidden_layers):
            self.register_parameter("posterior_w_means_" + str(i), posterior_w_means[i])
            self.register_parameter("posterior_w_log_vars_" + str(i), posterior_w_log_vars[i])
            self.register_parameter("posterior_b_means_" + str(i), posterior_b_means[i])
            self.register_parameter("posterior_b_log_vars_" + str(i), posterior_b_log_vars[i])

        for i in range(self.n_heads):
            self.register_parameter("head_posterior_w_means_" + str(i), head_posterior_w_means[i])
            self.register_parameter("head_posterior_w_log_vars_" + str(i), head_posterior_w_log_vars[i])
            self.register_parameter("head_posterior_b_means_" + str(i), head_posterior_b_means[i])
            self.register_parameter("head_posterior_b_log_vars_" + str(i), head_posterior_b_log_vars[i])

    def _mean_posterior_variance(self):
        """
        Return the mean posterior variance for logging purposes.
        Excludes the head layer.
        """
        ((_, posterior_w_log_vars), (_, posterior_b_log_vars)) = self.posterior
        posterior_log_vars = torch.cat([torch.reshape(t, (-1,)) for t in posterior_w_log_vars] + posterior_b_log_vars)
        posterior_vars     = torch.exp(posterior_log_vars)
        return torch.mean(posterior_vars).item()

In [9]:
#@title MultiHeadMLP

#from https://github.com/NixGD/variational-continual-learning and edited to make compatible with experiment loops and replay
class MultiHeadMLP(torch.nn.Module):
    def __init__(self, x_dim, h_dim, y_dim, n_heads=1, shared_h_dims=(100)):
        super().__init__()
        # check for bad parameters
        if n_heads < 1:
            raise ValueError('Network requires at least one head.')

        self.x_dim = x_dim
        self.h_dim = h_dim
        self.y_dim = y_dim
        self.n_heads = n_heads

        shared_dims = [x_dim] + list(shared_h_dims) + [h_dim] if type(shared_h_dims) == tuple else [x_dim] + [shared_h_dims] + [h_dim]

        # list of layers in shared network
        self.shared_layers = torch.nn.ModuleList([
            torch.nn.Linear(shared_dims[i], shared_dims[i + 1]) for i in
            range(len(shared_dims) - 1)
        ])

        self.heads = torch.nn.ModuleList([
            torch.nn.Linear(self.h_dim, self.y_dim) for _ in range(n_heads)
        ])

        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, x, head_idx):
        for layer in self.shared_layers:
            x = F.relu(layer(x))

        x = self.heads[head_idx](x)

        x = self.softmax(x)
        return x

    def prediction(self, x, head_idx):
        return torch.argmax(self(x, head_idx), dim=1)

    def point_estimate_loss(self, x, y, head):
        predictions = self(x, head)
        loss = torch.nn.NLLLoss()(predictions, y)
        return loss

    def vcl_loss(self, x, y, head_idx, task_length):
        return self.point_estimate_loss(x, y, head_idx)

    def reset_for_new_task(self, head):
        pass

In [10]:
#@title Coreset

#taken from https://github.com/NixGD/variational-continual-learning and edited, added kcenter method

class Coreset():
    """
    Base class for the the coreset.  This version of the class has no
    coreset but subclasses will replace the select method.
    """

    def __init__(self, size=0, lr=0.001):
        self.size = size
        self.coreset = None
        self.coreset_task_ids = None
        self.lr = lr

    def select(self, d: data.Dataset, task_id: int):
        """
        Given a torch dataset, will choose k datapoints.  Will then update
        the coreset with these datapoints.
        Returns: the subset that was not selected as a torch dataset.
        """

        return d

    def coreset_train(self, m, old_optimizer, tasks, epochs, device,
                      y_transform=None, multiheaded=True, batch_size=256):
        """
        Returns a new model, trained on the coreset.  The returned model will
        be a deep copy, except when coreset is empty (when it will be identical)

        tasks can be either a list, in which case the coreset will be trained
        on all tasks in the list, or an integer, in which case it will be
        trained on only that task.
        """

        if self.coreset is None:
            return m

        model = deepcopy(m)

        optimizer = optim.Adam(model.parameters(), lr=self.lr)
        optimizer.load_state_dict(old_optimizer.state_dict())

        # if tasks is an integer, turn it into a singleton.
        if isinstance(tasks, int):
            tasks = [tasks]

        #EDITTED debug
        print(f"Training on coreset of size {len(self.coreset)}")

        # create dict of train_loaders
        train_loaders = {
            task_idx :  data.DataLoader(
                            task_subset(self.coreset, self.coreset_task_ids, task_idx),
                            batch_size
                        )
            for task_idx in tasks
        }

        print('CORESET TRAIN')
        for _ in tqdm(range(epochs), 'Epochs: '):
            # Randomize order of training tasks
            shuffle(tasks)
            for task_idx in tasks:
                head = task_idx if multiheaded else 0

                for batch in train_loaders[task_idx]:
                    optimizer.zero_grad()
                    x, y_true = batch
                    x = x.to(device)
                    y_true = y_true.to(device)

                    if y_transform is not None:
                        y_true = y_transform(y_true, task_idx)

                    loss = model.vcl_loss(x, y_true, head, len(self.coreset))
                    loss.backward()
                    optimizer.step()

        return model


class RandomCoreset(Coreset):

    def __init__(self, size):
        super().__init__(size)

    def select(self, d : data.Dataset, task_id : int):

        new_cs_data, non_cs = data.random_split(d, [self.size, max(0,len(d)-self.size)])

        # Need to split the x from the y values to also include the task values.
        # I don't like this way of doing it, but I couldn't find something better.
        new_cs_x = torch.tensor([x for x, _ in new_cs_data])
        new_cs_y = torch.tensor([y for _, y in new_cs_data])

        new_cs = data.TensorDataset(new_cs_x, new_cs_y)
        new_task_ids = torch.full((len(new_cs_data),), task_id)

        if self.coreset is None:
            self.coreset = new_cs
            self.coreset_task_ids = new_task_ids
        else:
            self.coreset = data.ConcatDataset((self.coreset, new_cs))
            self.coreset_task_ids = torch.cat((self.coreset_task_ids, new_task_ids))

        return non_cs

#(ADDED K-CENTER CORESET
class kCenterCoreset(Coreset):

    def __init__(self, size):
        super().__init__(size)

    def select(self, d : data.Dataset, task_id : int):

        X = np.array([x for x, _ in d])
        Y = np.array([y for _, y in d])

        dists = np.full(X.shape[0], np.inf)
        current = 0
        best_indices = []
        dists = self.update_dist(dists, X, current)
        best_indices.append(current)

        for i in range(1, self.size):
          current = np.argmax(dists)
          dists = self.update_dist(dists, X, current)
          best_indices.append(current)

        non_best_indices = [i for i in range(X.shape[0]) if i not in best_indices]
        non_cs = data.Subset(d, non_best_indices)

        new_cs_x = torch.tensor(X[best_indices])
        new_cs_y = torch.tensor(Y[best_indices])

        new_cs = data.TensorDataset(new_cs_x, new_cs_y)
        new_task_ids = torch.full((new_cs_y.size(0),), task_id)

        if self.coreset is None:
            self.coreset = new_cs
            self.coreset_task_ids = new_task_ids
        else:
            self.coreset = data.ConcatDataset((self.coreset, new_cs))
            self.coreset_task_ids = torch.cat((self.coreset_task_ids, new_task_ids))

        return non_cs

    def update_dist(self, dists, X, current):
      for i in range(X.shape[0]):
        current_dists = np.linalg.norm(X[i,:]-X[current,:])
        dists[i] = np.minimum(current_dists, dists[i])
      return dists

In [11]:
#@title Replay

#all orginal code

class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return self.Y.size(0)

    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

class Replay():

  def __init__(self, metrics, size_calc_info, tau = 0.5, total_size_goal = 200, choose_per_batch_override = 0, lr=0.001, epochs_per_populate = 20, choices = 100):
    #size_calc_info formatted as (EPOCHS, N_TASKS, BATCH_SIZE, len(TRAIN_SUBSET))
    #"total_size_goal" is how many data points we want to select/add per task
    self.buffer = []
    self.tau = tau
    self.total_size_goal = total_size_goal
    self.total_size_real = 0
    self.replay_task_ids = None
    self.lr = lr
    self.metrics = metrics
    self.taskset_X = []
    self.taskset_Y = []
    self.epochs_per_populate = epochs_per_populate
    self.d = None
    self.choices = choices
    if choose_per_batch_override == 0:
      self.choose_per_batch = math.ceil(total_size_goal/choices)
    else:
      self.choose_per_batch = choose_per_batch_override
    self.sample_size = size_calc_info[2]
    print(f'choose per batch : {self.choose_per_batch}')#DEBUG
    print(f'sample size: {self.sample_size}')#DEBUG

  def _log_prob_replay(self, model, X, Y, head, num_samples = TRAIN_NUM_SAMPLES, y_transform=None):
    #TRAIN_NUM_SAMPLES is from VCL model (set to 10 orginally)
    print(f'COMPUTING LOG PROBS FOR REPLAY SELECTION')
    outputs = []
    n = Y.size(0)

    if y_transform is not None:
      Y = y_transform(Y, len(self.buffer))

    for i in range(num_samples):
      outputs.append(model.forward(X.to(DEVICE), head))

    losses = []
    Xl = torch.cat(outputs)
    Yl = Y.repeat(num_samples,1).view(-1)

    for i in range(n):
      indices = [num for num in range(n * num_samples) if num % n == i]
      loss = nn.CrossEntropyLoss()(Xl[indices].to(DEVICE), Yl[indices].to(DEVICE))
      losses.append(loss)

    return losses

  def compute_grads(self, model, X, Y, head, y_transform=None):

    S = "similarity" in self.metrics
    A = "affinity" in self.metrics

    if hasattr(model, "_calculate_kl_term"):
      loss = model._calculate_kl_term(head).cpu()
      params = filter(lambda p: p.requires_grad, model.parameters())
      kl_grad = parameters_to_vector(filter(lambda p: p != None, grad(loss, params, allow_unused=True, retain_graph=True)))
      n_params = kl_grad.size(0)
    else:
      #TEMP fix for getting n_params when model doesn't have a kl_term
      loss = model.vcl_loss(torch.tensor([X[0].tolist()]).to(DEVICE), torch.tensor([Y[0]]).to(DEVICE), 0, 1)
      params = filter(lambda p: p.requires_grad, model.parameters())
      n_params = parameters_to_vector(filter(lambda p: p != None, grad(loss, params, allow_unused=True, retain_graph=True))).size(0)
      kl_grad = torch.zeros(1, n_params)

    normalized_gradients = torch.zeros(Y.size(0), n_params)
    gradients = torch.zeros(Y.size(0), n_params) if S else None
    replay_gradient = torch.zeros(1, n_params) if A else None
    losses = self._log_prob_replay(model, X, Y, head, y_transform=y_transform)

    print("Computing loss gradients of each point in batch for replay score")
    for i in tqdm(range(Y.size(0)), "Data point (from batch): "):
      params = filter(lambda p: p.requires_grad, model.parameters())
      loss = losses[i]
      gradient = parameters_to_vector(filter(lambda p: p != None ,grad(loss, params, allow_unused=True, retain_graph=True))).to(DEVICE) + kl_grad.to(DEVICE) #added .to(DEVICE) (for pMNIST non VCL)
      if S:
        gradients[i,:] = gradient
      normalized_gradients[i,:] = gradient/torch.norm(gradient)

    if A:

      print("Computing loss gradients for replay sample")
      if len(self.buffer) == 0:
        replay_gradient = None
      else:
        for task_idx in range(len(self.buffer)):

          n_Y = self.buffer[task_idx]["Y"].size(0)
          if self.sample_size < n_Y:
            random_indices = torch.randperm(n_Y)[:self.sample_size]
            replay_X = (self.buffer[task_idx]["X"])[random_indices]
            replay_Y = (self.buffer[task_idx]["Y"])[random_indices]
          else:
            replay_X = self.buffer[task_idx]["X"]
            replay_Y = self.buffer[task_idx]["Y"]

          params = filter(lambda p: p.requires_grad, model.parameters())
          replay_losses = self._log_prob_replay(model, replay_X.float(), replay_Y.type(torch.LongTensor), head, y_transform=y_transform)
          replay_gradient[0,:] = (parameters_to_vector(filter(lambda p: p!=None, grad(replay_losses, params, allow_unused=True, retain_graph=True))).to(DEVICE) + kl_grad.to(DEVICE)) / len(replay_losses) #added .to(DEVICE) (for pMNIST non VCL)
          normalized_replay_gradient = replay_gradient / torch.norm(replay_gradient)

    if replay_gradient == None:
      normalized_replay_gradient = None

    return {'gradients' : gradients, "normalized_gradients" : normalized_gradients, "normalized_replay_gradient" : normalized_replay_gradient}

  def similarity(self, grads):

    gradients = grads['gradients']
    normalized_gradients = grads['normalized_gradients']
    batch_gradient = torch.sum(gradients, dim=0) / gradients.size(0)
    normalized_batch_gradient = batch_gradient / torch.norm(batch_gradient)
    return normalized_gradients @ normalized_batch_gradient

  def diversity(self, grads):

    normalized_gradients = grads['normalized_gradients']
    N = normalized_gradients.size(0)
    pairwise_prods = normalized_gradients @ normalized_gradients.t()
    pairwise_prods[range(N), range(N)] = 0
    diversity_unnorm = torch.sum(pairwise_prods, dim=1)
    return (-1/(N-1)) * diversity_unnorm

  def affinity(self, grads):

    normalized_gradients = grads['normalized_gradients']
    normalized_replay_gradient = grads['normalized_replay_gradient']
    n_params = normalized_gradients.size(0)
    if normalized_replay_gradient == None:
      return torch.zeros(1, n_params).squeeze()
    else:
      return (normalized_gradients @ normalized_replay_gradient.t()).squeeze()

  def populate_taskset(self, model, batch, head, y_transform=None):

    X, Y = batch
    self.d = X.size(1)
    n = Y.size(0)

    if self.choose_per_batch < n:

      scores = torch.zeros(n)
      grads = self.compute_grads(model, X, Y, head, y_transform=y_transform)

      if "similarity" in self.metrics:
        scores += self.similarity(grads)
      if "diversity" in self.metrics:
        scores += self.diversity(grads)
      if "affinity" in self.metrics:
        scores += self.tau * self.affinity(grads)

      best_indices = torch.topk(scores, k=self.choose_per_batch).indices
      best_X = X[best_indices]
      best_Y = Y[best_indices]

    else:
      print(f'No grad score computations, putting entire batch into buffer')
      best_X = X
      best_Y = Y

    self.taskset_X.append(best_X)
    self.taskset_Y.append(best_Y)

  def update_buffer(self):

    stack_X = torch.stack(self.taskset_X).view(-1, self.d)
    stack_Y = torch.stack(self.taskset_Y).view(-1)
    np_X = stack_X.numpy()
    np_Y = stack_Y.unsqueeze(1).numpy()
    np_XY_unique = np.unique(np.concatenate((np_X, np_Y), axis=1), axis=0)
    np_X_unique = np_XY_unique[:, :-1]
    np_Y_unique = np_XY_unique[:, -1]

    self.total_size_real += np_Y_unique.size
    self.buffer.append({'X': torch.tensor(np_X_unique), 'Y': torch.tensor(np_Y_unique)})
    self.taskset_X = []
    self.taskset_Y = []

  #PORTIONS OF BELOW CODE BORROWED FROM NIXGD
  def replay_train(self, m, old_optimizer, tasks, epochs, device,
                      y_transform=None, multiheaded=True, batch_size=256, neglect_first_task = True):


    if isinstance(tasks, int):
      tasks = [tasks]

    if neglect_first_task and len(tasks) == 1 and tasks[0] == 0:
      print('First task: replay buffer not used')
      return m

    model = deepcopy(m)

    optimizer = optim.Adam(model.parameters(), lr=self.lr)
    optimizer.load_state_dict(old_optimizer.state_dict())

    train_loaders = {}
    replay_len = 0
    for task_idx in tasks:

        n_Y = self.buffer[task_idx]["Y"].size(0)
        if self.total_size_goal < n_Y:
          random_indices = torch.randperm(n_Y)[:self.total_size_goal] #approx temp fix to sometimes over selecting replay samples by a tiny amount (batch size is not divsor of dataset size for example)
          X = (self.buffer[task_idx]["X"])[random_indices]
          Y = (self.buffer[task_idx]["Y"])[random_indices]
        else:
          X = self.buffer[task_idx]["X"]
          Y = self.buffer[task_idx]["Y"]
        dataset = CustomDataset(X, Y)
        train_loaders[task_idx] = data.DataLoader(dataset, batch_size)
        replay_len += len(dataset)

    print("REPLAY BUFFER TRAIN")
    print(f'Replay buffer size: {replay_len}')
    for _ in tqdm(range(epochs), 'Epochs: '):

      shuffle(tasks)
      for task_idx in tasks:
        head = task_idx if multiheaded else 0

        for batch in train_loaders[task_idx]:
          optimizer.zero_grad()
          x, y_true = batch
          x = x.to(device)
          y_true = y_true.to(device)

          if y_transform is not None:
              y_true = y_transform(y_true, task_idx)

          loss = model.vcl_loss(x.float().to(DEVICE), y_true.type(torch.LongTensor).to(DEVICE), head, replay_len) #added .to(DEVICE) (for pMNIST non VCL)
          loss.backward()
          optimizer.step()

    return model


In [12]:
#@title run_point_estimate_initialisation

# taken from https://github.com/NixGD/variational-continual-learning

def run_point_estimate_initialisation(model, data, epochs, task_ids, batch_size,
                                      device, lr, task_idx=0, y_transform=None,
                                      multiheaded=True):
    """
    Trains a VCL model in 'non-variational' mode on a task to learn good initial estimates
    for the model parameter means.

    :param model: the VCL model to train
    :param data: the complete dataset to train on, such as MNIST
    :param epochs: number of training epochs
    :param task_ids: the label-to-task mapping that defines which task examples in the dataset belong to
    :param batch_size: batch size used in training
    :param device: device to run the experiment on, either 'cpu' or 'cuda'
    :param lr: optimizer learning rate to use
    :param task_idx: task being learned, maps to a specific head in the network
    :param y_transform: transform to be applied to the dataset labels
    :param multiheaded: true if the network being trained is multi-headed
    """
    print("Obtaining point estimate for posterior initialisation")

    head = task_idx if multiheaded else 0

    # each task has its own optimizer
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # obtain the appropriate data subset depending on which task we are running
    task_data = task_subset(data, task_ids, task_idx)
    loader = DataLoader(task_data, batch_size)

    # train
    for _ in tqdm(range(epochs), 'Epochs: '):
        for batch in loader:
            optimizer.zero_grad()
            x, y_true = batch
            x = x.to(device)
            y_true = y_true.to(device)

            if y_transform is not None:
                y_true = y_transform(y_true, task_idx)

            loss = model.point_estimate_loss(x, y_true, head=head)
            loss.backward()
            optimizer.step()

In [13]:
#@title run_task

#inspired by https://github.com/NixGD/variational-continual-learning code, editted heavily to accomdate replay

#EDITED to include replay
def run_task(model, train_data, train_task_ids, test_data, test_task_ids,
             task_idx, coreset, replay, epochs, batch_size, save_as, device, lr,
             y_transform=None, multiheaded=True, train_full_coreset=True,
             summary_writer=None, log={"exp": "unknown_exp", "model": "unknown_model"}):
#
    """
        Trains a VCL model using online variational inference on a task, and performs a coreset
        training run as well as an evaluation after training.

        :param model: the VCL model to train
        :param train_data: the complete dataset to train on, such as MNIST
        :param train_task_ids: the label-to-task mapping that defines which task examples in the dataset belong to
        :param test_data: the complete dataset to train on, such as MNIST
        :param test_task_ids: the label-to-task mapping that defines which task examples in the dataset belong to
        :param task_idx: task being learned, maps to a specific head in the network
        :param coreset: coreset object to use in training
        :param epochs: number of training epochs
        :param batch_size: batch size used in training
        :param save_as: base directory to save into
        :param device: device to run the experiment on, either 'cpu' or 'cuda'
        :param lr: optimizer learning rate to use
        :param y_transform: transform to be applied to the dataset labels
        :param multiheaded: true if the network being trained is multi-headed
        :param summary_writer: tensorboard_x summary writer
        """

    print('TASK ', task_idx)

    # separate optimizer for each task
    optimizer = optim.Adam(model.parameters(), lr=lr)

    head = task_idx if multiheaded else 0

    # obtain correct subset of data for training, and set some aside for the coreset
    task_data = task_subset(train_data, train_task_ids, task_idx)
    non_coreset_data = coreset.select(task_data, task_id=task_idx)
    train_loader = DataLoader(non_coreset_data, batch_size)

    #EDITTED calc when to populate replay
    n_batches = math.ceil(len(non_coreset_data) / batch_size)
    n_steps = n_batches * epochs

    #EDITTED to accomodate replay
    replay_on = replay != None
    choose_step = n_steps // replay.choices if replay_on else 0

    # EDITTED for replay
    step = 0
    for epoch in tqdm(range(epochs), 'Epochs: '):
        epoch_loss = 0

        for batch in train_loader:

            optimizer.zero_grad()
            x, y_true = batch
            x = x.to(device)
            y_true = y_true.to(device)

            if y_transform is not None:
                y_true = y_transform(y_true, task_idx)

            loss = model.vcl_loss(x, y_true, head, len(task_data))
            epoch_loss += len(x) * loss.item()

            replay_time = (step+1) % choose_step == 0 or step == 0 if replay_on else False

            if replay_time:
              replay.populate_taskset(model, batch, head, y_transform=y_transform)

            loss.backward()
            optimizer.step()
            step += 1

        # if summary_writer is not None:
        #     summary_writer.add_scalars("loss", {"TASK_" + str(task_idx): epoch_loss / len(task_data)}, epoch)

    if replay_on:
      replay.update_buffer()
    #

    # after training, prepare for new task by copying posteriors into priors
    model.reset_for_new_task(head)

    # train using full coreset
    if train_full_coreset:
        model_cs_trained = coreset.coreset_train(
            model, optimizer, list(range(task_idx + 1)), epochs,
            device, y_transform=y_transform, multiheaded=multiheaded)

    #EDITED to incorporate replay
    if replay_on:
          model_cs_trained = replay.replay_train(
            model, optimizer, list(range(task_idx + 1)), epochs,
            device, y_transform=y_transform, multiheaded=multiheaded)
    #

    # test
    task_accuracies = []
    tot_right = 0
    tot_tested = 0

    for test_task_idx in range(task_idx + 1):
        if not train_full_coreset:
            model_cs_trained = coreset.coreset_train(
                model, optimizer, test_task_idx, epochs,
                device, y_transform=y_transform, multiheaded=multiheaded)

        head = test_task_idx if multiheaded else 0

        task_data = task_subset(test_data, test_task_ids, test_task_idx)

        x = torch.Tensor([x for x, _ in task_data])
        y_true = torch.Tensor([y for _, y in task_data])
        x = x.to(device)
        y_true = y_true.to(device)

        if y_transform is not None:
            y_true = y_transform(y_true, test_task_idx)

        y_pred = model_cs_trained.prediction(x, head)

        acc = class_accuracy(y_pred, y_true)
        print("After task {} perfomance on task {} is {}"
              .format(task_idx, test_task_idx, acc))

        tot_right += acc * len(task_data)
        tot_tested += len(task_data)
        task_accuracies.append(acc)

    mean_accuracy = tot_right / tot_tested
    print("Mean accuracy:", mean_accuracy)

    #EDITED log results

    # data_dir = "/content/drive/MyDrive/UDL MP/data"
    # if not os.path.exists(data_dir):
    #   os.makedirs(data_dir)

    # exp_dir = data_dir + "/" + log["exp"]
    # if not os.path.exists(exp_dir):
    #   os.makedirs(exp_dir)

    # run_dir = exp_dir + "/" + log['model']
    # if not os.path.exists(run_dir):
    #   os.makedirs(run_dir)

    file_data = {"task": task_accuracies, "mean": mean_accuracy}

    dir = "/content/drive/MyDrive/UDL MP/data/" + log["exp"] + "/" + log["model"] #CHANGE to your dir path
    if not os.path.exists(dir):
      os.makedirs(dir)

    file_path = dir + "/" + f"Task {task_idx+1}"

    with open(file_path, 'w') as json_file:
      json.dump(file_data, json_file)

    # if summary_writer is not None:
    #     task_accuracies_dict = dict(zip(["TASK_" + str(i) for i in range(task_idx + 1)], task_accuracies))
    #     summary_writer.add_scalars("test_accuracy", task_accuracies_dict, task_idx + 1)
    #     summary_writer.add_scalar("mean_posterior_variance", model._mean_posterior_variance(), task_idx + 1)
    #     summary_writer.add_scalar("mean_accuracy", mean_accuracy, task_idx + 1)

#EDITTTTTTTT below
    #write_as_json(save_as + '/accuracy.txt', task_accuracies)
    #save_model(model, save_as + '_model_task_' + str(task_idx) + '.pth')

In [14]:
#@title Permuted MNIST run

#inspired by https://github.com/NixGD/variational-continual-learning code, editted heavily to accomdate replay

MNIST_FLATTENED_DIM = 28 * 28
LR = 0.001
INITIAL_POSTERIOR_VAR = 1e-3

def permuted_mnist(
    #EDTIED to move make this vars arguements rather than just method vars
    N_CLASSES = 10,
    LAYER_WIDTH = 100,
    N_HIDDEN_LAYERS = 2,
    N_TASKS = 10,
    MULTIHEADED = False,
    CORESET_SIZE = 200,
    EPOCHS = 100,
    BATCH_SIZE = 256,
    TRAIN_FULL_CORESET = True,
    #EDITED to add subset of MNIST selection functionality
    TRAIN_SUBSET = [i for i in range(6000)], #[random.randint(0, 59999) for _ in range(1000)]
    TEST_SUBSET = [i for i in range(1000)], #[random.randint(0, 9999) for _ in range(1000)]
    #
    #EDITED to add coreset/replay selection functionality
    CORESET_TYPE = 'none', #choose between 'none', 'center', 'random'
    REPLAY_TYPE = [], #choose between [], and a list populated with any combination of "similarity", "diversity", and "affinity"
    REPLAY_SIZE = 200,
    VCL_ON = True,
    LOG = {"exp": "unknown_exp", "model": "unknown_model"},
    CPBO = 0
    ):

    """
    Runs the 'Permuted MNIST' experiment from the VCL paper, in which each task
    is obtained by applying a fixed random permutation to the pixels of each image.
    """

    # flattening and permutation used for each task
    transforms = [Compose([Flatten(), Scale(), Permute(torch.randperm(MNIST_FLATTENED_DIM))]) for _ in range(N_TASKS)]

    if VCL_ON:

      model = DiscriminativeVCL(
          in_size=MNIST_FLATTENED_DIM, out_size=N_CLASSES,
          layer_width=LAYER_WIDTH, n_hidden_layers=N_HIDDEN_LAYERS,
          n_heads=(N_TASKS if MULTIHEADED else 1),
          initial_posterior_var=INITIAL_POSTERIOR_VAR
      ).to(DEVICE)

    else:
      BATCH_SIZE = CORESET_SIZE
      model = MultiHeadMLP(784, LAYER_WIDTH, 10).to(DEVICE)

    #EDITED to add coreset type
    if CORESET_TYPE == 'none':
      coreset = Coreset()
    elif CORESET_TYPE == "random":
      coreset = RandomCoreset(size=CORESET_SIZE)
    elif CORESET_TYPE == 'center':
      coreset = kCenterCoreset(size=CORESET_SIZE)
    #

    #EDITED to add replay
    if REPLAY_TYPE == []:
      replay = None
    else:
      len_train = len(TRAIN_SUBSET) if TRAIN_SUBSET != None else 60000
      size_calc_info = (EPOCHS, N_TASKS, BATCH_SIZE, len_train)
      replay = Replay(REPLAY_TYPE, size_calc_info, total_size_goal = REPLAY_SIZE) if CPBO == 0 else Replay(REPLAY_TYPE, size_calc_info, total_size_goal = REPLAY_SIZE, choose_per_batch_override=CPBO)
    #

    #EDITED to add subset functionality
    if TRAIN_SUBSET == None:
      mnist_train = ConcatDataset(
          [MNIST(root='./data', train=True, download=True, transform=t) for t in transforms]
      )

    else:
      mnist_train = ConcatDataset(
          [torch.utils.data.Subset(MNIST(root='./data', train=True, download=True, transform=t), TRAIN_SUBSET) for t in transforms]
      )

    task_size = len(mnist_train) // N_TASKS
    train_task_ids = torch.cat(
        [torch.full((task_size,), id) for id in range(N_TASKS)]
    )

    if TEST_SUBSET == None:
      mnist_test = ConcatDataset(
          [MNIST(root='./data', train=False, download=True, transform=t) for t in transforms]
      )

    else:
      mnist_test = ConcatDataset(
          [torch.utils.data.Subset(MNIST(root='./data', train=False, download=True, transform=t), TEST_SUBSET) for t in transforms]
      )

    task_size = len(mnist_test) // N_TASKS
    test_task_ids = torch.cat(
        [torch.full((task_size,), id) for id in range(N_TASKS)]
    )
    #

    #summary_logdir = os.path.join("logs", "disc_p_mnist", datetime.now().strftime('%b%d_%H-%M-%S'))
    #writer = SummaryWriter(summary_logdir)
    #EDITED to not include os calls
    writer = SummaryWriter()
    #

    run_point_estimate_initialisation(model=model, data=mnist_train,
                                      epochs=EPOCHS, batch_size=BATCH_SIZE,
                                      device=DEVICE, lr=LR,
                                      multiheaded=MULTIHEADED,
                                      task_ids=train_task_ids)

    # each task is classification of MNIST images with permuted pixels
    #EDITED to add replay
    for task in range(N_TASKS):
        run_task(
            model=model, train_data=mnist_train, train_task_ids=train_task_ids,
            test_data=mnist_test, test_task_ids=test_task_ids, task_idx=task,
            coreset=coreset, replay=replay, epochs=EPOCHS, batch_size=BATCH_SIZE,
            device=DEVICE, lr=LR, save_as="disc_p_mnist",
            multiheaded=MULTIHEADED, train_full_coreset=TRAIN_FULL_CORESET,
            summary_writer=writer,log=LOG
        )

    #DEBUG
    if replay != None:
      for task in range(N_TASKS):
        rx = replay.buffer[task]["X"]
        ry = replay.buffer[task]["Y"]
        print(f'Replay buffer X for task {task} is {rx}')
        print(f'Replay buffer Y for task {task} is {ry}')
    #

    writer.close()


In [15]:
#@title Split MNIST run

#inspired by https://github.com/NixGD/variational-continual-learning code, editted heavily to accomdate replay
def split_mnist(
    #EDTIED to move make this vars arguements and the accomodate subset, coreset type, and replay
    N_CLASSES = 2,
    LAYER_WIDTH = 256,
    N_HIDDEN_LAYERS = 2,
    N_TASKS = 5,
    MULTIHEADED = True,
    CORESET_SIZE = 40,
    EPOCHS = 100, #they used 120 with no minibatching, we try 100 epoch with minibatching so we dont have to compute grads of whole dataset for replay
    BATCH_SIZE = 256,
    TRAIN_FULL_CORESET = True,
    TRAIN_SUBSET = [i for i in range(6000)],
    TEST_SUBSET = [i for i in range(1000)],
    CORESET_TYPE = 'none', #choose between 'none', 'center', 'random'
    REPLAY_TYPE = [], #choose between [], and a list populated with any combination of "similarity", "diversity", and "affinity"
    REPLAY_SIZE = 200,
    VCL_ON = True,
    LOG = {"exp": "unknown_exp", "model": "unknown_model"}
#
):
    """
    Runs the 'Split MNIST' experiment from the VCL paper, in which each task is
    a binary classification task carried out on a subset of the MNIST dataset.
    """

    transform = Compose([Flatten(), Scale()])

    # download dataset
    #EDITED to add subset
    mnist_train = data.Subset(MNIST(root='./data', train=True, download=True, transform=transform), TRAIN_SUBSET) if TRAIN_SUBSET != None else MNIST(root='./data', train=True, download=True, transform=transform)
    mnist_test = data.Subset(MNIST(root='./data', train=False, download=True, transform=transform), TEST_SUBSET) if TEST_SUBSET != None else MNIST(root='./data', train=False, download=True, transform=transform)
    #

    #EDITED to accomondate VCL off mode
    if VCL_ON:

      model = DiscriminativeVCL(
          in_size=MNIST_FLATTENED_DIM, out_size=N_CLASSES,
          layer_width=LAYER_WIDTH, n_hidden_layers=N_HIDDEN_LAYERS,
          n_heads=(N_TASKS if MULTIHEADED else 1),
          initial_posterior_var=INITIAL_POSTERIOR_VAR
      ).to(DEVICE)

    else:
      model = MultiHeadMLP(784, LAYER_WIDTH, 10, n_heads = 5).to(DEVICE)


    #EDITED to add coreset type
    if CORESET_TYPE == 'none':
      coreset = Coreset()
    elif CORESET_TYPE == "random":
      coreset = RandomCoreset(size=CORESET_SIZE)
    elif CORESET_TYPE == "center":
      coreset = kCenterCoreset(size=CORESET_SIZE)
    #

    # #EDITED to add replay
    # if REPLAY_TYPE == []:
    #   replay = None
    # else:
    #   train_len = len(TRAIN_SUBSET) if TRAIN_SUBSET != None else 60000
    #   size_calc_info = (EPOCHS, N_TASKS, BATCH_SIZE, train_len)
    #   replay = Replay(REPLAY_TYPE, size_calc_info, choose_per_batch_override = 2)
    # #
    #EDITED to add replay
    if REPLAY_TYPE == []:
      replay = None
    else:
      len_train = len(TRAIN_SUBSET) if TRAIN_SUBSET != None else 60000
      size_calc_info = (EPOCHS, N_TASKS, BATCH_SIZE, len_train)
      replay = Replay(REPLAY_TYPE, size_calc_info, total_size_goal = REPLAY_SIZE)
    #

    label_to_task_mapping = {
        0: 0, 1: 0,
        2: 1, 3: 1,
        4: 2, 5: 2,
        6: 3, 7: 3,
        8: 4, 9: 4,
    }

    if isinstance(mnist_train[0][1], int):
        train_task_ids = torch.Tensor([label_to_task_mapping[y] for _, y in mnist_train])
        test_task_ids = torch.Tensor([label_to_task_mapping[y] for _, y in mnist_test])
    elif isinstance(mnist_train[0][1], torch.Tensor):
        train_task_ids = torch.Tensor([label_to_task_mapping[y.item()] for _, y in mnist_train])
        test_task_ids = torch.Tensor([label_to_task_mapping[y.item()] for _, y in mnist_test])

    # summary_logdir = os.path.join("logs", "disc_s_mnist", datetime.now().strftime('%b%d_%H-%M-%S'))
    # writer = SummaryWriter(summary_logdir)
    #EDITED to not include os calls
    writer = SummaryWriter()
    #

    # each task is a binary classification task for a different pair of digits
    binarize_y = lambda y, task: (y == (2 * task + 1)).long()

    run_point_estimate_initialisation(model=model, data=mnist_train,
                                      epochs=EPOCHS, batch_size=BATCH_SIZE,
                                      device=DEVICE, multiheaded=MULTIHEADED,
                                      lr=LR, task_ids=train_task_ids,
                                      y_transform=binarize_y)

    for task_idx in range(N_TASKS):
        run_task(
            model=model, train_data=mnist_train, train_task_ids=train_task_ids,
            test_data=mnist_test, test_task_ids=test_task_ids, coreset=coreset, replay=replay,
            task_idx=task_idx, epochs=EPOCHS, batch_size=BATCH_SIZE, lr=LR,
            save_as="disc_s_mnist", device=DEVICE, multiheaded=MULTIHEADED,
            y_transform=binarize_y, train_full_coreset=TRAIN_FULL_CORESET,
            summary_writer=writer, log=LOG
        )

    #DEBUG
    if replay != None:
      for task in range(N_TASKS):
        rx = replay.buffer[task]["X"]
        ry = replay.buffer[task]["Y"]
        print(f'Replay buffer X for task {task} is {rx}')
        print(f'Replay buffer Y for task {task} is {ry}')
    #

    writer.close()

In [ ]:
#EXAMPLE

#to run VCL with replay (similarity score) on permuted MNIST, run:

permuted_mnist(REPLAY_TYPE=['similarity'], LOG = {'exp': "Name of your experiment", "model": "VCL with replay (similarity)"})